In [2]:
import numpy as np
import pandas as pd

In [3]:
caInputeDF = pd.read_csv("ConjointInput.csv", sep = ";")

Input = a ranking of 3 different features (TV Size, TV Type, TV Color) with 3(32",37",42")<br>
, 2 (LCD, Plasma), 3(Black, Silver, Anthrazit) different stimuli types.

In [64]:

caInputeDF

,Stimulus,Rank
0,A1B1C1,2
1,A1B1C2,3
2,A1B1C3,1
3,A1B2C1,5
4,A1B2C2,6
5,A1B2C3,4
6,A2B1C1,8
7,A2B1C2,9
8,A2B1C3,7
9,A2B2C1,11


## First step is to introduce dummy variables for every stimulus<p>
There are in total 9 differen stimuli, and 18 different combinations


In [6]:
ConjointDummyDF = pd.DataFrame(np.zeros((18,9)), columns=["Rank","A1", "A2", "A3",
                                                    "B1","B2", 
                                                    "C1", "C2",
                                                    "C3"])

In [7]:
ConjointDummyDF.Rank = caInputeDF.Rank

for index, row in caInputeDF.iterrows(): 
    stimuli1, stimuli2, stimuli3 = caInputeDF["Stimulus"].ix[index][:2], \
    caInputeDF["Stimulus"].ix[index][2:4], caInputeDF["Stimulus"].ix[index][4:6]
    
    
    ConjointDummyDF.ix[index, [stimuli1,stimuli2,stimuli3]] = 1

In [8]:
ConjointDummyDF.head()

,Rank,A1,A2,A3,B1,B2,C1,C2,C3
0,2,1,0,0,1,0,1,0,0
1,3,1,0,0,1,0,0,1,0
2,1,1,0,0,1,0,0,0,1
3,5,1,0,0,0,1,1,0,0
4,6,1,0,0,0,1,0,1,0


## Insert the proper Stimulus names

In [9]:
fullNames = {"Rank":"Rank", \
           "A1": "32\" (81cm)","A2": "37\" (94cm)","A3": "42\" (107cm)", \
          "B1": "Plasma", "B2":"LCD", \
           "C1":"Silver", "C2":"Black", "C3": "Anthrazit",\
          }

ConjointDummyDF.rename(columns=fullNames, inplace=True)

In [10]:
ConjointDummyDF.head()

,Rank,"32"" (81cm)","37"" (94cm)","42"" (107cm)",Plasma,LCD,Silver,Black,Anthrazit
0,2,1,0,0,1,0,1,0,0
1,3,1,0,0,1,0,0,1,0
2,1,1,0,0,1,0,0,0,1
3,5,1,0,0,0,1,1,0,0
4,6,1,0,0,0,1,0,1,0


## Estimate Main Effects with a linear regression
<p>

There are different ways for parameter estimation beside linear regression depending on what kind of rating you have.<br>
For example using Probit or Logit is the output is not a rank but a decision (1=chose stimulus, 0 = no choice).

In [11]:
import statsmodels.api as sm

In [12]:
ConjointDummyDF.columns

Index([u'Rank', u'32" (81cm)', u'37" (94cm)', u'42" (107cm)', u'Plasma',
       u'LCD', u'Silver', u'Black', u'Anthrazit'],
      dtype='object')

In [13]:
X = ConjointDummyDF[[u'32" (81cm)', u'37" (94cm)', u'42" (107cm)', u'Plasma',\
       u'LCD', u'Silver', u'Black', u'Anthrazit']]
X = sm.add_constant(X)
Y = ConjointDummyDF.Rank
linearRegression = sm.OLS(Y, X). fit()
linearRegression.summary()

C:\Users\Janis\Anaconda2\lib\site-packages\scipy\stats\stats.py:1277: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   Rank   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 4.763e+30
Date:                Wed, 16 Dec 2015   Prob (F-statistic):          2.40e-181
Time:                        11:47:37   Log-Likelihood:                 572.69
No. Observations:                  18   AIC:                            -1133.
Df Residuals:                      12   BIC:                            -1128.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
const           4.3846   4.91e-16   8.94e+15      0.000         4.385     4.385
32" (81cm)     -4.5385   1.51e-15     -3e+15      0.000        -4.538    -4.538
37" (94cm)      1.4615   1.51e-15   9.66e+14      0.000         1.462     1.462
42" (107cm)     7.4615   1.51e-15   4.93e+15      0.000         7.462     7.462
Plasma          0.6923   1.09e-15   6.34e+14      0.000         0.692     0.692
LCD             3.6923   1.09e-15   3.38e+15      0.000         3.692     3.692
Silver          1.4615   1.51e-15   9.66e+14      0.000         1.462     1.462
Black           2.4615   1.51e-15   1.63e+15      0.000         2.462     2.462
Anthrazit       0.4615   1.51e-15   3.05e+14      0.000         0.462     0.462
==============================================================================
Omnibus:                        2.660   Durbin-Watson:                   0.228
Prob(Omnibus):                  0.264   Jarque-Bera (JB):                1.985
Skew:                           0.669   Prob(JB):                        0.371
Kurtosis:                       2.075   Cond. No.                     2.12e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 8.68e-32. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

## Part worth values & relative importance of the stimuli
<p>
Importance of Stimuli= Max(beta) - Min(beta)
<br>
Relative Importance of Stimuli = Importance of Stim / Sum(Importance of all Stimuli)

In [48]:
importance = []
relative_importance = []

rangePerFeature = []

begin = "A"
tempRange = []
for stimuli in fullNames.keys():
    if stimuli[0] == begin:
        tempRange.append(linearRegression.params[fullNames[stimuli]])
    elif stimuli == "Rank":
        rangePerFeature.append(tempRange)
    else:
        rangePerFeature.append(tempRange)
        begin = stimuli[0]
        tempRange = [linearRegression.params[fullNames[stimuli]]]
        

In [49]:
for item in rangePerFeature:
    importance.append( max(item) - min(item))

for item in importance:
    relative_importance.append(100* round(item/sum(importance),3))

### Summary & Results

In [55]:
print "Relative Importance of Feature:\n\nMonitor Size:",relative_importance[0], "%",\
"\nType of Monitor:", relative_importance[1], "%", "\nColor of TV:", relative_importance[2], "%"

Relative Importance of Feature:

Monitor Size: 70.6 % 
Type of Monitor: 17.6 % 
Color of TV: 11.8 %


What would be the optimal product bundle? <p>
42", LCD, Black

In [62]:
#As array that looks like X
#Must include Constant!

optBundle = [1,0,0,1,0,1,0,1,0]
print "The best possible Combination of Stimuli would have the highest rank:",\
linearRegression.predict(optBundle)[0]

The best possible Combination of Stimuli would have the highest rank: 18.0
